In [1]:
import os 
import streamlit
# from langchain_groq import ChatGroq
# from langchain_community.document_loaders import WebBaseLoader
# from langchain_community.embeddings import OllamaEmbeddings
# from langchain_community.vectorstores import FAISS
# from langchain_core.prompts import ChatPromptTemplate
# from langchain.chains.combine_documents import create_stuff_documents_chain
# from langchain.chains import create_retrieval_chain
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain.chains import RetrievalQA
# from llama_index.llms.groq import Groq 
import dotenv 
dotenv.load_dotenv()

True

In [2]:

from huggingface_hub import login
from llama_index.llms.together import TogetherLLM
# llm=ChatGroq(
#     groq_api_key=os.getenv('GROQ_API_KEY'),
#     model_name='llama-3.1-70b-versatile',
#     temperature=0.2,
    
    
# )
llm=TogetherLLM(
    api_key=os.getenv('TOGETHER_API_KEY'),
    model='meta-llama/Llama-3-70b-chat-hf'
    # model='meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo'
)
# llm=Groq(
#     api_key=os.getenv('GROQ_API_KEY'),
#     model='llama-3.1-70b-versatile'
# )
login(token=os.getenv('HUGGINGFACEHUB_API_TOKEN'),add_to_git_credential=True)

d:\anaconda\envs\genai\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# split the document and store it 

In [18]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, load_index_from_storage, StorageContext
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.embeddings.together import TogetherEmbedding
from llama_index.embeddings.openai import OpenAIEmbedding
# import faiss
# embeddings=OllamaEmbeddings()
# loader=WebBaseLoader('https://en.wikipedia.org/wiki/Elon_Musk')
# docs=loader.load()
# text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
# documents=text_splitter.split_documents(docs)
# print(documents)

documents=SimpleDirectoryReader('../data/').load_data()
# embed_model=HuggingFaceEmbedding(
#     model_name='BAAI/bge-small-en-v1.5'
# )
embed_model=TogetherEmbedding(
    model_name=os.getenv('TOGETHERAI_EMBEDDING_MODEL_NAME'),
    api_key=os.getenv('TOGETHER_API_KEY')
)
# embed_model = OpenAIEmbedding(model="text-embedding-3-large")
from llama_index.core import Settings

Settings.chunk_size=1024
Settings.embed_model=embed_model
Settings.llm=llm
print(documents)

[Document(id_='bb95728c-bb1a-4deb-ab8c-53a40fdd0454', embedding=None, metadata={'page_label': '1', 'file_name': 'texasLaw.pdf', 'file_path': 'd:\\pythonProjects\\RAG_finetuning\\explore\\..\\data\\texasLaw.pdf', 'file_type': 'application/pdf', 'file_size': 5534498, 'creation_date': '2024-11-07', 'last_modified_date': '2024-11-07'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text="38 REPORT OF ATTORNEY GENERAL.\nOPINIONS.\nAs being of general importance or interest the following opinions have-\nbeen selected for publication from the large number rendered:\nCommissions of County Treasurer.-County treasurer is entitled to receive a com-\nmission of one-half of one per cent on money coming into his hands as permanent\nschoolfund of a county. and is also entit

# store or retrive the data from database

In [4]:
# index=VectorStoreIndex.from_documents(documents=documents,embed_model=Settings.embed_model)
# # save the data 
# index.storage_context.persist('../data/')
# # load index from storage
# storage_context=StorageContext.from_defaults(persist_dir='../data/')
# index=load_index_from_storage(storage_context)

# # create a retriever or query engine 
# retriever=index.as_retriever()
# query_engine=index.as_query_engine()


In [5]:
from llama_index.core import StorageContext
from llama_index.vector_stores.chroma import ChromaVectorStore
import chromadb
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from IPython.display import Markdown, display

# create chroma collection
db=chromadb.PersistentClient(path='../data/chroma_db/') 
# chroma_client=chromadb.EphemeralClient()
chroma_collection=db.get_or_create_collection('test')

# set up chromavector store and load in the data
# embed_model=LangchainEmbedding(HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2'))
vector_store=ChromaVectorStore(chroma_collection=chroma_collection)

storage_context=StorageContext.from_defaults(vector_store=vector_store)
index=VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, embed_model=embed_model
)

query_engine=index.as_query_engine()
response=query_engine.query('summarize the info')
display(Markdown(f"<b>{response}</b>"))




<b>The provided information appears to be excerpts from a legal document, specifically related to labor laws in Texas. The content discusses the scope of laborers' lien laws, clarifying that they only apply to individuals who perform personal labor, such as mechanics, laborers, and operatives, and do not extend to contractors who furnish the labor of others or provide tools and teams. The text also references specific court cases and legislative amendments that support this interpretation.</b>

In [6]:
import torch
torch.cuda.is_available()

True

In [7]:
print(chroma_collection.get()['documents'])

["38 REPORT OF ATTORNEY GENERAL.\nOPINIONS.\nAs being of general importance or interest the following opinions have-\nbeen selected for publication from the large number rendered:\nCommissions of County Treasurer.-County treasurer is entitled to receive a com-\nmission of one-half of one per cent on money coming into his hands as permanent\nschoolfund of a county. and is also entitled to a like commission for disbursing\nthis fund. CountU treasurer is also entitled to receive commissions on the pro-\nceeds of countU bonds issued for the purpose of making improvements and turned\nover to the contractor.\nATTORNEY GENERAL'S OFFICE.\nAU$TIN. December 10. 1892.\nAlexander Coker, Esq.. County Treasurer, Oakville. Texas.\nDEARt SIR-Replying to your favor of December S., I bag to state that the\nquestions you ask have already been considered in reply to a letter from thecounty judge of your county. To reiterate. you are entitle(] to charge coniuis-\nsions of one-half of one per cent on money 

In [8]:
# load from db
from llama_index.core import StorageContext
from llama_index.vector_stores.chroma import ChromaVectorStore
import chromadb
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from IPython.display import Markdown, display

db2=chromadb.PersistentClient(path='../data/chroma_db') 
chroma_collection=db2.get_or_create_collection('test')
print(chroma_collection.get()['documents'])
# embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")
vector_store=ChromaVectorStore(chroma_collection=chroma_collection)
index=VectorStoreIndex.from_vector_store(
    vector_store,
    embed_model=embed_model,
)
query_engine=index.as_query_engine()
response=query_engine.query('summarize the info')
display(Markdown(f"<b>{response}</b>"))

["38 REPORT OF ATTORNEY GENERAL.\nOPINIONS.\nAs being of general importance or interest the following opinions have-\nbeen selected for publication from the large number rendered:\nCommissions of County Treasurer.-County treasurer is entitled to receive a com-\nmission of one-half of one per cent on money coming into his hands as permanent\nschoolfund of a county. and is also entitled to a like commission for disbursing\nthis fund. CountU treasurer is also entitled to receive commissions on the pro-\nceeds of countU bonds issued for the purpose of making improvements and turned\nover to the contractor.\nATTORNEY GENERAL'S OFFICE.\nAU$TIN. December 10. 1892.\nAlexander Coker, Esq.. County Treasurer, Oakville. Texas.\nDEARt SIR-Replying to your favor of December S., I bag to state that the\nquestions you ask have already been considered in reply to a letter from thecounty judge of your county. To reiterate. you are entitle(] to charge coniuis-\nsions of one-half of one per cent on money 

<b>The provided information appears to be related to the laborers' lien law in Texas. It discusses the purpose and scope of the law, which is to secure wages for mechanics, laborers, and operatives who have performed work on a road. The law does not extend to work done by others, nor to the use of teams or tools not personally used by the individual claiming wages. The legislature has amended the law to include tools and teams, but it remains restricted to personal services and does not apply to contractors who furnish the labor of others.</b>

# chat

In [9]:
query_engine=index.as_query_engine()
response=query_engine.query('what does the document talk about?')
display(Markdown(f"<b>{response}</b>"))

<b>The document appears to be a collection of legal opinions and discussions related to various aspects of Texas law, including labor laws, land ownership, and insurance company taxation.</b>

In [10]:
## add prompts 

In [11]:
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.core.prompts import ChatPromptTemplate
from llama_index.core.memory import BaseMemory
# memory=BaseMemory()
language= 'French'
# Text QA Prompt
chat_text_qa_msgs = [
    ChatMessage(
        role=MessageRole.SYSTEM,
        content=(
            """ 
            You are an experienced immigration lawyer providing detailed legal advice in {language}. 
            Carefully review the following documents to extract relevant information and address the client's question 
            Analyze any legal regulations, risks, or implications that may apply, and provide guidance on the next steps.

                Instructions:
                1. Identify key legal issues or relevant regulations from the provided documents.
                2. Explain how the information applies to the client’s situation.
                3. Highlight any significant risks, legal obligations, or considerations the client should be aware of.
                4. Offer clear advice on the next steps, considering short-term and long-term outcomes.
                5. Note any additional documentation, evidence, or forms the client should prepare

                Please provide a structured and thorough response, clearly addressing each point in a way that helps the client understand their legal standing and options in {language}.
            """
        ),
    ),
    ChatMessage(
        role=MessageRole.USER,
        content=(
            "Context information is below.\n"
            "---------------------\n"
            "{context_str}\n"
            "---------------------\n"
            "Given the context information and not prior knowledge, "
            "answer the query.\n"
            "Query: {query_str}\n"
            "Answer: "
        ),
    ),
]

text_qa_template = ChatPromptTemplate(chat_text_qa_msgs)

# Refine Prompt
chat_refine_msgs = [
    ChatMessage(
        role=MessageRole.SYSTEM,
        content=(
            "You are an expert Q&A system that strictly operates in two modes "
            "when refining existing answers:\n"
            "1. **Rewrite** an original answer using the new context.\n"
            "2. **Repeat** the original answer if the new context isn't useful.\n"
            "Never reference the original answer or context directly in your answer.\n"
            "When in doubt, just repeat the original answer."
        ),
    ),
    ChatMessage(
        role=MessageRole.USER,
        content=(
            "New Context: {context_msg}\n"
            "Query: {query_str}\n"
            "Original Answer: {existing_answer}\n"
            "New Answer: "
        ),
    ),
]
refine_template = ChatPromptTemplate(chat_refine_msgs)

In [12]:
from llama_index.core.memory import ChatMemoryBuffer

memory= ChatMemoryBuffer.from_defaults(token_limit=1000)
chat_engine=index.as_chat_engine(
    chat_mode='context',
    memory=memory,
    system_prompt=(text_qa_template,refine_template),
)


In [13]:
response=chat_engine.chat('what was the latest law passed?')
print(response.response)

Based on the provided context information, it appears that the documents are excerpts from the Texas Law report, specifically from the Office of the Attorney General. However, there is no specific mention of the "latest law passed" in the provided context.

The first excerpt discusses the duties of the county clerk in recording instruments in the deed records and releasing vendor's liens. The second excerpt explains the ministerial duties of county, district, and State canvassers in determining election results.

Without additional information or context, it is not possible to determine the latest law passed. If you could provide more context or clarify the specific law or time period you are referring to, I would be happy to try and assist you further.


In [14]:
response=chat_engine.chat('could you provide more info')
print(response.response)

Based on the provided context information, it appears that the documents are excerpts from the Texas Law report, specifically from the Office of the Attorney General.

The first excerpt, dated January 18, 1894, discusses the authority of the Board of University Regents to use matriculation fees for paying salaries, including increasing the salary of the provost from $1200 to $1500. The Attorney General's opinion is that the Board of Regents has the authority to fix salaries and use matriculation fees for this purpose, as long as it is in accordance with the general object of the law.

The second excerpt appears to be discussing a laborers' lien law, specifically the object and purpose of the statute, which is to secure wages due to mechanics, laborers, and operatives for work and labor done in constructing, repairing, or operating roads. The law does not extend to work and labor done by others, nor to the use of teams or tools and implements other than those personally used by the pers

In [15]:
response=chat_engine.chat('what are the things i sould avoid?')
print(response.response)

Based on the provided context information, it appears that the documents are excerpts from the Texas Law report, specifically from the Office of the Attorney General.

From the excerpts, it can be inferred that one should avoid the following:

1. Fraudulent activities in the administration of land laws, as mentioned in the report of the Attorney General on page 37.

2. Misinterpretation of the laborers' lien law, specifically attempting to extend it to work and labor done by others, or to the use of teams or tools and implements other than those personally used by the person claiming wages.

3. Failure to comply with the requirements for patenting land, including occupancy and residence as a home for a period of three consecutive years, as mentioned in the report of the Attorney General on page 37.

4. Non-compliance with tax laws, specifically the annual State tax on insurance companies, as mentioned in the report of the Attorney General on page 74.

It is essential to carefully revie

In [16]:
response=chat_engine.chat('anything perticular about investing?')
print(response.response)

Based on the provided context information, there is no specific mention of investing or any particular guidance related to investing. The excerpts appear to be discussing various aspects of Texas law, including land laws, laborers' lien laws, and tax laws, but do not provide any specific advice or guidance on investing.

However, it is worth noting that the second excerpt mentions the act of 1837 and the amendment of 1893, which appears to be related to land sales and forfeiture. It states that upon default in the payment of interest, the land becomes forfeited to the State and reverts to the particular fund to which it originally belonged, subject to be resold under the act or any future law.

In general, when investing in land or real estate, it is essential to carefully review the relevant laws and regulations, including those related to land sales, forfeiture, and taxation, to ensure that you are aware of any potential risks or obligations. It is also crucial to conduct thorough du

In [17]:
response=chat_engine.chat('tell me more about taxes and what should i avoid?')
print(response.response)

Based on the provided context information, it appears that the documents are excerpts from the Texas Law report, specifically from the Office of the Attorney General.

Regarding taxes, the excerpts mention the following:

1. Insurance companies are subject to an annual State tax of one and one-fourth per cent on the gross amount of premiums received in this State during the preceding year (page 74).
2. Fire, marine, health, live stock, guarantee, and accident insurance companies are subject to an annual State tax of one-half of one per cent on the gross amount of premiums received in this State during the preceding year (page 74).
3. All insurance companies are subject to State, county, and municipal ad valorem taxes upon their real and personal property within the State (page 74).
4. There is no occupation tax levied on insurance companies by any county, city, or town (page 74).

To avoid potential tax-related issues, it is essential to:

1. Ensure compliance with all applicable tax l